# Import all the Dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras import models , layers 
import matplotlib.pyplot as plt 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


# Set all the Constents

In [ ]:
BATCH_SIZE = 32 
IMAGE_SIZE = 256 
CHANNELS = 3
EPOCHS = 50 

# Import data into tensorflow dataset

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "/kaggle/input/plantvillagepotato/PlantVillage",
   seed=123 ,
    shuffle=True,
   image_size = (IMAGE_SIZE , IMAGE_SIZE ),
   batch_size = BATCH_SIZE 
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
for image_batch , label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

# Visualize some of the images for our datasets

In [ ]:
plt.figure(figsize=(10 ,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3 ,4 , i + 1) 
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(class_names[label_batch[i]])
      

# Function to Split Dataset
Dataset should be bifurcated into 3 subsets, namely:

1. Training: Dataset to be used while training
2. Validation: Dataset to be tested against while training
3. Test: Dataset to be tested against after we trained a model

In [ ]:
def get_dataset_partitions_tf(ds , train_split=0.8 , val_split=0.1 , test_split=0.1 , shuffle=True , shuffle_size=10000):
    assert(train_split + val_split + test_split ) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size , seed=123)
    
    train_size = int( train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds , val_ds , test_ds 

In [ ]:
train_ds, val_ds , test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
print("Len Train_ds: {} , Len Val_ds: {} , Len Test_ds: {}".format(len(train_ds), len(val_ds), len(test_ds)))

# Cache,Shuffle and Prefetch the Dataset

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds =  val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Building the Model
**Creating a Layer for Resizing and Normalization**

Before we feed our images to network, we should be resizing it to the desired size. Moreover, to improve model performance, we should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256). This should happen while training as well as inference. Hence we can add that as a layer in our Sequential Model.

You might be thinking why do we need to resize (256,256) image to again (256,256). You are right we don't need to but this will be useful when we are done with the training and start using the model for predictions. At that time somone can supply an image that is not (256,256) and this layer will resize it

In [ ]:
resizing_rescaling = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE , IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1/255.)
    ])

# Data Augmention

Data Augmentation is needed when we have less data, this boosts the accuracy of our model by augmenting the data.

In [ ]:
data_augmention = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

# Applying Data Augmentaion for training data

In [ ]:
train_ds = train_ds.map(
lambda x , y : (data_augmention( x , training=True ) , y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model Architecture

We use a CNN coupled with a Softmax activation in the output layer. We also add the initial layers for resizing, normalization and Data Augmentation.

**We are going to use convolutional neural network (CNN) here. CNN is popular for image classification tasks. Watch below video to understand fundamentals of CNN**

In [ ]:
input_shape = (BATCH_SIZE , IMAGE_SIZE , IMAGE_SIZE , CHANNELS)
n_classes = 3

model = tf.keras.Sequential([
    resizing_rescaling , 
    layers.Conv2D(32 , kernel_size=(3,3) , activation='relu' , input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu' ),
    layers.MaxPooling2D((2,2)) ,
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu' ),
    layers.MaxPooling2D((2,2)) ,
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu' ),
    layers.MaxPooling2D((2,2)) ,
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu' ),
    layers.MaxPooling2D((2,2)) ,
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu' ),
    layers.MaxPooling2D((2,2)) ,
    layers.Flatten(),
    layers.Dense(64 , activation='relu'),
    layers.Dense(n_classes , activation='softmax'),
])
model.build(input_shape=input_shape)

In [ ]:
model.summary()

# Compiling the Model

We use adam Optimizer, SparseCategoricalCrossentropy for losses, accuracy as a metric

In [ ]:
model.compile(
    optimizer="adam" ,
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    )

In [ ]:
history = model.fit(train_ds , batch_size = BATCH_SIZE , validation_data=val_ds , verbose=1 , epochs= EPOCHS)